In [1]:
import pickle
from dataclasses import dataclass
from typing import Any, Callable

import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import fmin
from tqdm.auto import tqdm
from math import sin, cos
from poom.optimizers.newton.utils import Visualizer
from utils.helpers import make_mesh


In [2]:
class Obj:

    def f(self, x):
        A1 = 0.5 * sin(1) - 2 * cos(1) + sin(2) - 1.5 * cos(2)
        A2 = 1.5 * sin(1) - cos(1) + 2 * sin(2) - 0.5 * cos(2)
        B1 = 0.5 * sin(x[0]) - 2 * cos(x[0]) + sin(x[1]) - 1.5 * cos(x[1])
        B2 = 1.5 * sin(x[0]) + cos(x[0]) + 2 * sin(x[1]) - 0.5 * cos(x[1])
        return (1 + (A1 - B1)**2 + (A2 - B2)**2)


    def g(self, x):
        return ((x[0] + 3)**2) + (x[1] + 1)**2

    def Fs(self, x):
        return np.array([self.f(x), self.g(x)])

    def Fss(self):
        return np.array([self.f, self.g])


# class Obj:
#     def f(self, x):
#         return x[0]**2 + 3 * (x[1] - 1)**2 + (x[2] - 2)**2

#     def g(self, x):
#         return 2 * (x[0] - 1)**2 + x[1]**2

#     def h(self, x):
#         return 2 * (x[0] - 1)**2 + x[2]**2

#     def Fs(self, x):
#         return np.array([self.f(x), self.g(x), self.h(x)])

#     def Fss(self):
#         return np.array([self.f, self.g, self.h])

In [3]:
# contrained conditions

@dataclass
class SteepestDescent:
    ndim: int
    nu: float #alpha
    sigma: float #m1
    eps: float

    def grad(self, f, x, h=1e-4):
        g = np.zeros_like(x)
        for i in range(self.ndim):
            tmp = x[i]
            x[i] = tmp + h
            yr = f(x)
            x[i] = tmp - h
            yl = f(x)
            g[i] = (yr - yl) / (2 * h)
            x[i] = tmp
        return g

    def nabla_F(self, x):
        obj = Obj()
        F = obj.Fss()
        nabla_F = np.zeros((len(F), self.ndim)) # (m, n) dimensional matrix
        for i, f in enumerate(F):
            nabla_F[i] = self.grad(F[i], x)
        return nabla_F

    def phi(self, d, x):
        nabla_F = self.nabla_F(x)
        return max(np.dot(nabla_F, d)) + 0.5 * np.linalg.norm(d) ** 2

    def theta(self, d, x):
        return self.phi(d, x) + 0.5 * np.linalg.norm(d) ** 2

    def armijo(self, d, x):
        obj = Obj()
        t = 1
        Fl = np.array(obj.Fs(x + t * d))
        Fr = np.array(obj.Fs(x))
        Re = self.sigma * t * np.dot(self.nabla_F(x), d)
        while np.all(Fl > Fr + Re):
            t *= self.nu
            Fl = np.array(obj.Fs(x + t * d))
            Fr = np.array(obj.Fs(x))
            Re = self.sigma * t * np.dot(self.nabla_F(x), d)
        return t
    
    def steepest(self, x):
        obj = Obj()
        list_point = []
        d = np.array(fmin(self.phi, x, args=(x, ), disp=False))
        th = self.theta(d, x)
        for i in range(50):
            th = self.theta(d, x)
            t = self.armijo(d, x)
            y = obj.Fs(x)
            list_point.append(y)
            x = x + t * d
            d = np.array(fmin(self.phi, x, args=(x, ), disp=False))
            if abs(th) < self.eps:
              break
        return list_point

In [5]:
sd = SteepestDescent(
    ndim=2,
    nu=0.5,
    sigma=0.01,
    eps=1e-8,
)

res = []
# run
x_init = 2*np.random.rand(400, 2)
for x in tqdm(x_init):
    ans = np.array(sd.steepest(x))
    res.append(ans)

  0%|          | 0/400 [00:00<?, ?it/s]

In [59]:
obj = Obj()
n_objectives = 2

In [19]:
visualizer = Visualizer(results=res, n_objectives=n_objectives)

In [60]:
with open("res_steepest_2.pl", "wb") as f:
    pickle.dump(res, f)

In [55]:
len(res[0])

12